In [37]:
import pandas as pd
import numpy as np
import ta
import talib
import plotly.graph_objects as go

import datetime

import util_functions as uf

In [3]:
sma_value = [50]
close='Close'
sym='AAPL'

today = datetime.datetime.now()
end_date = datetime.datetime(today.year, today.month, today.day-1)
start_date = datetime.datetime(today.year-1, 1, 1)

In [177]:
df = uf.getdata(sym, sma_value, close, start_date, end_date)

[*********************100%***********************]  1 of 1 completed


In [77]:
df = df.dropna(subset=['sma_50']).reset_index(drop='TRUE')

In [80]:
SMAsignal = [0]*len(df)
backcandles = 15

for row in range(backcandles, len(df)):
    upt = 1
    dnt = 1
    for i in range(row-backcandles, row+1):
        if max(df['Open'][i], df[close][i])>=df['sma_50'][i]:
            dnt=0
        if min(df['Open'][i], df[close][i])<=df['sma_50'][i]:
            upt=0

    if upt==1:
        SMAsignal[row]=2
    elif dnt==1:
        SMAsignal[row]=1

df['SMASignal'] = SMAsignal

In [81]:
def isPivot(candle, window):
    """
    function that detects if a candle is a pivot/fractal point
    args: candle index, window before and after candle to test if pivot
    returns: 1 if pivot high, 2 if pivot low, 3 if both and 0 default
    """
    if candle-window < 0 or candle+window >= len(df):
        return 0
    
    pivotHigh = 1
    pivotLow = 2
    for i in range(candle-window, candle+window+1):
        if df.iloc[candle]['Low'] > df.iloc[i]['Low']:
            pivotLow=0
        if df.iloc[candle]['High'] < df.iloc[i]['High']:
            pivotHigh=0
    if (pivotHigh and pivotLow):
        return 3
    elif pivotHigh:
        return pivotHigh
    elif pivotLow:
        return pivotLow
    else:
        return 0

In [179]:
df['isPivot'] = df.apply(lambda x: isPivot(x.name,window=3), axis=1)

In [180]:
def pointpos(x):
    if x['isPivot']==2:
        return x['Low']-3
    elif x['isPivot']==1:
        return x['High']+3
    else:
        return np.nan
df['pointpos'] = df.apply(lambda row: pointpos(row), axis=1)

In [ ]:
def detect_structure(candle, backcandles, window, close, zone_width):
    """
    Attention! window should always be greater than the pivot window! to avoid look ahead bias
    """
    if (candle <= (backcandles+window)) or (candle+window+1 >= len(df)):
        return 0
    
    localdf = df.iloc[candle-backcandles-window:candle-window] #window must be greater than pivot window to avoid look ahead bias
    highs = localdf[localdf['isPivot'] == 1]['High'].values
    lows = localdf[localdf['isPivot'] == 2]['Low'].values
    levelbreak = 0
    
    if len(lows)==3:
        support_condition = True
        mean_low = lows.mean()
        for low in lows:
            if abs(low-mean_low)>zone_width:
                support_condition = False
                break
        if support_condition and (mean_low - df.loc[candle][close])>zone_width:
            levelbreak = 1

    if len(highs)==3:
        resistance_condition = True
        mean_high = highs.mean()
        for high in highs:
            if abs(high-mean_high)>zone_width:
                resistance_condition = False
                break
        if resistance_condition and (df.loc[candle][close]-mean_high)>zone_width:
            levelbreak = 2
    return levelbreak

In [194]:
df['pattern_detected'] = df.apply(lambda row: detect_structure(row.name, backcandles=30, window=6, close='Close', zone_width=5), axis=1)

In [195]:
localdf = df.iloc[39-30-6:39-6]
highs = localdf[localdf['isPivot'] == 1]['High'].values


In [196]:
print(highs)
print(highs.mean())

[147.22999573 157.38000488 156.33000183]
153.64666748046875


In [197]:
df[df['pattern_detected']!=0]

,Date,Open,High,Low,Close,Volume,ticker,sma_50,isPivot,pointpos,pattern_detected,pointpos_pattern
61,2023-03-31 00:00:00-04:00,162.440002,165.000000,161.910004,164.899994,68749800,AAPL,151.462799,0,NaN,2,NaN
62,2023-04-03 00:00:00-04:00,164.270004,166.289993,164.220001,166.169998,56976200,AAPL,152.028799,0,NaN,2,NaN
63,2023-04-04 00:00:00-04:00,166.600006,166.839996,165.110001,165.630005,46278300,AAPL,152.519200,1,169.839996,2,NaN
64,2023-04-05 00:00:00-04:00,164.740005,165.050003,161.800003,163.759995,51511700,AAPL,152.943799,0,NaN,2,NaN
65,2023-04-06 00:00:00-04:00,162.429993,164.960007,162.000000,164.660004,45390100,AAPL,153.399799,0,NaN,2,NaN
85,2023-05-05 00:00:00-04:00,170.979996,174.300003,170.759995,173.570007,113316400,AAPL,159.740800,1,177.300003,2,NaN
86,2023-05-08 00:00:00-04:00,172.479996,173.850006,172.110001,173.500000,55962800,AAPL,160.276600,0,NaN,2,NaN
87,2023-05-09 00:00:00-04:00,173.050003,173.539993,171.600006,171.770004,45326900,AAPL,160.753600,0,NaN,2,NaN
88,2023-05-10 00:00:00-04:00,173.020004,174.029999,171.899994,173.559998,53724500,AAPL,161.276600,0,NaN,2,NaN
89,2023-05-11 00:00:00-04:00,173.850006,174.589996,172.169998,173.750000,49514700,AAPL,161.845400,1,177.589996,2,NaN


In [198]:
def pointpos_pattern(x):
    if x['pattern_detected']==2:
        return x['High']+3
    elif x['pattern_detected']==1:
        return x['Low']-3
    else:
        return np.nan
df['pointpos_pattern'] = df.apply(lambda row: pointpos_pattern(row), axis=1)

In [199]:
fig = go.Figure(data=[go.Candlestick(x=df.index,
                open=df['Open'],
                high=df['High'],
                low=df['Low'],
                close=df['Close'])])

fig.add_scatter(x=df.index, y=df['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple", symbol='circle'),
                name="pivot")
fig.add_scatter(x=df.index, y=df['pointpos_pattern'], mode="markers",
                marker=dict(size=8, color="orange", symbol='arrow'),
                name="pivot")
fig.update_layout(xaxis_rangeslider_visible=False)

fig.show()

